# Supervised Learning Classifier by Naive Bayes

In [89]:
import glob
import codecs
import numpy
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score


pd.set_option('display.max_rows', 20, 
              'display.max_columns', 100, 
              'display.float_format', '{:,.2f}'.format)

In [70]:
a = '000085369099'
len(a)

12

In [71]:
a[-8:-2]

'853690'

# Getdata from database

In [72]:
# Connect Database
import pyodbc
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=YO-PC\SQLEXPRESS2016;"
                      "Database=ECS_DATA2;"
                      "Trusted_Connection=yes;")

cursor = conn.cursor()

In [73]:
df = pd.read_sql_query("SELECT * FROM Deci_InvoiceD", conn)
print(df.shape)
df.sample(5)

(23625, 121)


,BranchCode,RefNO,InvNO,ItemNO,DecItemNO,TariffCode,TariffSeq,StatCode,ImportTariff,PrivilegeCode,AHTN,DepositReson,IsFreeOfChage,UNDG,GroupCode,PdtCode,PdtSubCode,PdtDescriptionT,PdtDescriptionE,RTCProductCode,ProductAttribute1,ProductAttribute2,ProductYear,BrandName,DRemark,DOriginCountry,NetWeight,GrossWeight,DWeightUnit,TariffQty,TariffUnit,SalesCur,DegreeAmt,SalesRate,SalesPrice,SalesPriceTHB,SalesPackUnit,SalesPackQty,SalesTotalPrice,DSalesNetPriceTHB,IncreasedPrice,IncreasedCur,IncreasedRate,IncreasedPriceTHB,DSalesCIFPrice,DSalesCIFPriceTHB,CIFValueAssess,DShippingMark,PackAmount,PackUnit,...,ExpFin,ExpLnd,ExpOth,DExpShippingTHB,DExpFreightTHB,DExpInsuranceTHB,DExpPkgTHB,DExpFinTHB,DExpLndTHB,DExpOthTHB,SubPack,RawCode,ExpOth2,ExpOth3,ExpOth4,ExpOth5,DExpOthTHB2,DExpOthTHB3,DExpOthTHB4,DExpOthTHB5,AHTN2,AHTN3,AHTN4,AHTN5,RTCAHTN,ImpTaxIncentive,ArgTariffCode,ArgTariffSeq,ArgPrivilegeCode,OriginCer,ArgReasonCode,CerExportNo,ACDDDeductedAmt,ACDDProCode,ACDDValueCode,ACDDGrossW,ModelNumber,ModelVersion,ModelCmpTax,ArgValueRate,ArgSpcRate,ArgSpcCode,BOILicenseNo,Royalty,DetailRoyalty,OtherRemark,SubQTY,SortItemNo,SortDecItemNo,DetailOther
16237,00,DSYPI00001341,SVCHZ20180925006,16,16,000085371019,60041,000,,000,,,0,,IMPORT,EBR86133701,IM0001,แผงวงจรไฟฟ้า,"PCB ASSEMBLY,POWER\r\nEBR86133701",,,,2018,NO BRAND,,CN,0.83,0.00,KGM,1.00,C62,USD,0.00,32.70,7.34,240.04,EA,1.00,7.34,240.04,0.00,USD,32.70,0.00,10.69,349.59,0.00,SVCHZ20180925006,0.00,PX,...,0.00,0.00,0.00,0.00,107.26,2.29,0.00,0.00,0.00,0.00,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,,,,,,,,,,,0.00,,,0.00,,,,0.00,0.00,,,0,0,,,0,0,
18141,00,DSYPI00001509,LED20181015A,1,1,000085312000,60045,000,,000,,,0,,IMPORT,6327EC3008S,IM0001,หน้าจอแสดงผล แสดงออกมาเป็นตัวเลข,DISPLAY\r\n6327EC3008S,,,,2018,LG,,CN,305.85,0.00,KGM,"25,272.00",C62,USD,0.00,32.70,0.53,17.39,EA,"25,272.00","13,439.65","439,508.81",0.00,USD,32.70,0.00,"13,744.67","449,483.70",0.00,LG\r\nLED20181015A\r\nPALLET NO:1-4/4,4.00,PX,...,0.00,0.00,0.00,0.00,"5,580.01","4,394.88",0.00,0.00,0.00,0.00,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,,,,,,,,,,,0.00,,,0.00,,,,0.00,0.00,,,0,0,,,0,0,
1819,00,DSYP000000100,BP1810338,6,6,000084149091,60084,000,,ACN,,,0,,IMPORT,MAA62584911,IM0001,ส่วนประกอบคอมเพรสเซอร์,ACCUMULATOR\r\nMAA62584911,,,,2018,NO BRAND,,CN,"1,714.00",0.00,KGM,"3,544.00",C62,USD,0.00,32.30,1.47,47.49,C62,"3,544.00","5,209.68","168,292.98",0.00,USD,32.30,0.00,"5,440.27","175,741.94",0.00,N/A,0.00,PX,...,0.00,0.00,0.00,0.00,"5,765.28","1,683.68",0.00,0.00,0.00,0.00,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,,,,,,,,,,,0.00,,,0.00,,,,0.00,0.00,,,0,0,,,0,0,None
3465,00,DSYP000000256,SVCHQ20180618057,28,28,000085371019,60041,000,,000,,,0,,01,EBT62113501,01,แผงวงจรไฟฟ้า,CHASSIS ASSEMBLY\r\nEBT62113501,,,,0,NO BRAND,,KR,10.94,0.00,KGM,8.00,C62,USD,0.00,32.30,19.69,636.06,EA,8.00,157.52,"5,088.51",0.00,USD,32.30,0.00,157.52,"5,088.51",0.00,LG,0.00,CT,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,,,,,,,,,,,0.00,,,0.00,,,,0.00,0.00,,,0,0,,,0,0,
11999,00,DSYPI00000924,SVCHQ20180820056,346,346,000085299091,60051,000,,000,,,0,,IMPORT,COV34247901,IM0001,อุปกรณ์ประกอบทีวี,"BRACKET ASSEMBLY,OUTSOURCING\r\nCOV34247901",,,,2018,LG,,KR,0.01,0.00,KGM,1.00,C62,USD,0.00,33.11,0.10,3.31,EA,1.00,0.10,3.31,0.00,USD,33.11,0.00,0.10,3.31,0.00,LG,0.00,CT,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,,,,,,,,,,,,0.00,,,0.00,,,,0.00,0.00,,,0,0,,,0,0,


In [74]:
df = df[['TariffCode', 'PdtDescriptionE']]

df

,TariffCode,PdtDescriptionE
0,000085322900,CAPACITOR
1,000085322900,CAPACITOR
2,000085322900,CAPACITOR
3,000085369099,SOCKET
4,000085369099,SOCKET
...,...,...
23620,000054075200,TEXTILE PIECE GOODS\r\n95% POLYESTER 5% SPANDE...
23621,000054075200,TEXTILE PIECE GOODS\r\n95% POLYESTER 5% SPANDE...
23622,000054075200,TEXTILE PIECE GOODS\r\n95% POLYESTER 5% SPANDE...
23623,000054075200,TEXTILE PIECE GOODS\r\n100% POLYESTER\r\n75D*7...


In [75]:
data = df
data['length'] = data['TariffCode'].apply(len)
data.head()

,TariffCode,PdtDescriptionE,length
0,000085322900,CAPACITOR,12
1,000085322900,CAPACITOR,12
2,000085322900,CAPACITOR,12
3,000085369099,SOCKET,12
4,000085369099,SOCKET,12


In [76]:
print(data.length.min())
print(data.length.max())
print(data.length.unique())
data[data.length==11]

8
12
[12  8 11]


,TariffCode,PdtDescriptionE,length
92,'0000852990,"LCD,MODULE-TFT\r\nEAJ64411501",11


In [77]:
def range_txt(txt):
    
    if len(txt) == 12:
        tariff = txt[4:10]
        return tariff
    elif len(txt) == 11:
        tariff = '0' + txt[4:9]
        return tariff
    else:
        tariff = txt[0:6]
        return tariff



In [78]:
df['Code'] = df.TariffCode.apply(range_txt)

df

,TariffCode,PdtDescriptionE,length,Code
0,000085322900,CAPACITOR,12,853229
1,000085322900,CAPACITOR,12,853229
2,000085322900,CAPACITOR,12,853229
3,000085369099,SOCKET,12,853690
4,000085369099,SOCKET,12,853690
...,...,...,...,...
23620,000054075200,TEXTILE PIECE GOODS\r\n95% POLYESTER 5% SPANDE...,12,540752
23621,000054075200,TEXTILE PIECE GOODS\r\n95% POLYESTER 5% SPANDE...,12,540752
23622,000054075200,TEXTILE PIECE GOODS\r\n95% POLYESTER 5% SPANDE...,12,540752
23623,000054075200,TEXTILE PIECE GOODS\r\n100% POLYESTER\r\n75D*7...,12,540752


In [79]:
db = []
for i in range(len(df)):
    
    txt = df.PdtDescriptionE[i].strip()
    txt = txt.splitlines(keepends=False)
    
    db.append({'descript': txt ,'class':df.Code[i] })
    
db

[{'descript': ['CAPACITOR'], 'class': '853229'},
 {'descript': ['CAPACITOR'], 'class': '853229'},
 {'descript': ['CAPACITOR'], 'class': '853229'},
 {'descript': ['SOCKET'], 'class': '853690'},
 {'descript': ['SOCKET'], 'class': '853690'},
 {'descript': ['SOCKET'], 'class': '853690'},
 {'descript': ['SOCKET'], 'class': '853690'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '847149'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '847149'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '841490'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '841490'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '841490'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '841490'},
 {'descript': ['PUMP-DRAIN [DC31-00030A]'], 'class': '841490'},
 {'descript': ['LED MODULE', 'LED MODULE'], 'class': '853710'},
 {'descript': ['LED MODULE                                        ',
   'LED MODULE'],
  'class': '853710'},
 {'descript': ['LED MODULE                      

In [80]:
len(db)

23625

# Get data from master

In [90]:
df =pd.read_excel('data/input.xlsx',sheet_name='Sheet1', parse_string =['code'])
#df2 =pd.read_excel('data/HS-Code_target.xlsx',sheet_name='level_6', parse_string =['Code'])
#df = pd.concat([df1,df2], axis=0) 
df.head()

,code,description
0,010111,"Horses; live, pure-bred breeding animals"
1,010119,"Horses; live, other than pure-bred breeding an..."
2,010120,"Asses, mules and hinnies; live"
3,010210,"Bovine animals; live, pure-bred breeding animals"
4,010290,"Bovine animals; live, other than pure-bred bre..."


In [91]:
len(df)

76617

In [92]:
data = df[['code','description']]
data.head()

,code,description
0,010111,"Horses; live, pure-bred breeding animals"
1,010119,"Horses; live, other than pure-bred breeding an..."
2,010120,"Asses, mules and hinnies; live"
3,010210,"Bovine animals; live, pure-bred breeding animals"
4,010290,"Bovine animals; live, other than pure-bred bre..."


In [93]:
data.describe()

,code,description
count,76617,76617
unique,6554,15471
top,845090,WASHING MACHINE
freq,10585,2736


In [95]:
db_2 = []
for i in range(len(data)):
    
    db_2.append({'descript': data.description[i],'class':data.code[i] })
    
db_2


[{'descript': 'Horses; live, pure-bred breeding animals', 'class': '010111'},
 {'descript': 'Horses; live, other than pure-bred breeding animals',
  'class': '010119'},
 {'descript': 'Asses, mules and hinnies; live', 'class': '010120'},
 {'descript': 'Bovine animals; live, pure-bred breeding animals',
  'class': '010210'},
 {'descript': 'Bovine animals; live, other than pure-bred breeding animals',
  'class': '010290'},
 {'descript': 'Swine; live, pure-bred breeding animals', 'class': '010310'},
 {'descript': 'Swine; live, (other than pure-bred breeding animals), weighing less than 50kg',
  'class': '010391'},
 {'descript': 'Swine; live, (other than pure-bred breeding animals), weighing 50kg or more',
  'class': '010392'},
 {'descript': 'Sheep; live', 'class': '010410'},
 {'descript': 'Goats; live', 'class': '010420'},
 {'descript': 'Poultry; live, fowls of the species gallus domesticus, weighing not more than 185g',
  'class': '010511'},
 {'descript': 'Poultry; live, weighing not more

In [96]:
len(db_2)

76617

In [88]:
db

In [99]:
#convert เป็น DataFrame
#db = db.append(db_2)
data = pd.DataFrame(db_2)
print(len(data))
data

76617


,descript,class
0,"Horses; live, pure-bred breeding animals",010111
1,"Horses; live, other than pure-bred breeding an...",010119
2,"Asses, mules and hinnies; live",010120
3,"Bovine animals; live, pure-bred breeding animals",010210
4,"Bovine animals; live, other than pure-bred bre...",010290
...,...,...
76612,COMPRESSOR,841430
76613,COMPRESSOR,841430
76614,COMPRESSOR,841430
76615,WASHING MACHINE,845020


In [100]:
#แต่จริงแล้วดึงมาเฉพาะ field text

arr_text=data.iloc[:]['descript'].values

In [101]:
#CountVectorizer ทำ tokenize

vect = CountVectorizer(stop_words='english', lowercase=True)
vect

CountVectorizer(stop_words='english')

In [102]:
#เริ่มตัดคำ

count_train = vect.fit(arr_text)
count_train

CountVectorizer(stop_words='english')

In [103]:
#ดู vocabulary_

vect.vocabulary_

{'horses': 8212,
 'live': 8820,
 'pure': 10864,
 'bred': 3767,
 'breeding': 3769,
 'animals': 3255,
 'asses': 3363,
 'mules': 9941,
 'hinnies': 8175,
 'bovine': 3732,
 'swine': 11999,
 'weighing': 12854,
 '50kg': 1237,
 'sheep': 11490,
 'goats': 7921,
 'poultry': 10724,
 'fowls': 7734,
 'species': 11717,
 'gallus': 7813,
 'domesticus': 5083,
 '185g': 316,
 'ducks': 5148,
 'geese': 7847,
 'turkeys': 12545,
 'guinea': 8011,
 'chapter': 4124,
 'meat': 9406,
 'carcasses': 3990,
 'half': 8057,
 'fresh': 7754,
 'chilled': 4161,
 'cuts': 4774,
 'bone': 3705,
 'excluding': 7431,
 'boneless': 3706,
 'frozen': 7763,
 'hams': 8069,
 'shoulders': 11523,
 'thereof': 12281,
 'item': 8495,
 '0203': 36,
 'lamb': 8663,
 'including': 8345,
 'offal': 10117,
 'edible': 7250,
 'tongues': 12376,
 'livers': 8822,
 'cut': 4771,
 'pieces': 10511,
 'fatty': 7530,
 'rabbits': 10956,
 'hares': 8089,
 'frogs': 7759,
 'legs': 8727,
 'fat': 7528,
 'pig': 10514,
 'free': 7746,
 'lean': 8715,
 'rendered': 11106,
 'sal

In [104]:
#ทำ transform ให้อยู่ในรูปแบบ One Hot Encoding

transformer = vect.transform(arr_text)

In [105]:
#ดูผลลัพท์ matrix ที่ได้โดย array จะประกอบด้วย context ใน 1 paragraph หากมี word ใน context ให้ค่าเป็น 1 และหากไม่มี word ใน context ให้ค่าเป็น 0

print(transformer.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [106]:
#กลับมาเข้าเรื่องเดิมอีกครั้ง เตรียม tf-idf

from sklearn.feature_extraction.text import TfidfTransformer
# Config tf-idf

tfidf_transformer = TfidfTransformer(smooth_idf=False,).fit(transformer)

In [107]:
#show ค่า idf

tfidf_transformer.idf_

#tf(t) = จำนวนที่ปรากฏใน document/จำนวนคำทั้งหมดใน document
#idf(d, t) = ln(จำนวน document ทั้งหมด/จำนวนที่ปรากฏใน Document) + 1
#idf(d, t) = ln [ (1 + จำนวน document) / (1 + จำนวนที่ปรากฏใน Document) ] + 1
#Parameter smoot=true => ln((1+5)/(1+1))+1 = 2.09861229
#Parameter smoot=false=>ln(5)+1 = 2.60943791
#บางนิยามใช้ log() บางนิยามใช้ ln()

#เตรียมnaivebay

array([10.16713272, 11.55342708, 10.8602799 , ...,  9.94398917,
        9.94398917,  9.94398917])

In [108]:
from sklearn.naive_bayes import MultinomialNB

#เตรียม Class ในรูปแบบ array เหมือน arr_text

arr_class=data.iloc[:]['class'].values
arr_class

array(['010111', '010119', '010120', ..., '841430', '845020', '841582'],
      dtype=object)

In [109]:
#เตรียม train จาก transformer

messages_tfidf = tfidf_transformer.transform(transformer)
print (messages_tfidf.shape)

(76617, 13004)


In [110]:
messages_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [111]:
#สร้าง Model

detect_model = MultinomialNB().fit(messages_tfidf,arr_class)

In [112]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split( messages_tfidf,arr_class, test_size=0.4 , random_state=0)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

detect_model =MultinomialNB().fit(x_train,y_train)
#detect_model = MultinomialNB().fit(messages_tfidf,arr_class)

(45970, 13004) (30647, 13004) (45970,) (30647,)


In [113]:
print(detect_model.score(x_test, y_test))

0.4475805135902372


In [114]:
#การ Predict
#สร้าง ตัว test
#จะ test ด้วยคำว่า london bridge is falling down

sample = 'Sculptures and statuary'
arr_text[100]

'Fish; yellowfin tunas (thunnus albacares), frozen (excluding fillets, livers, roes and other fish meat of heading no. 0304)'

In [121]:
arr_text[100] =vect.transform([sample])

In [122]:
#แปลงเป็น vector

arr_text[100].toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [123]:
print(arr_text[100])

  (0, 11389)	1
  (0, 11818)	1


In [124]:
#นำไป Predict

print ('Predicted: ',detect_model.predict(arr_text[100]) )

Predicted:  ['845090']


In [125]:
#เห็นว่าคำทำนายออกมาเป็น london ตรงกับ Expected
#นำ data ทั้งหมดที่ train ไป test มันก็ควรจะตรง(Expected คือ 100%)

all_predictions = detect_model.predict(arr_text[100])
print(all_predictions)

['845090']


In [126]:
all_predictions[-1]

'845090'

In [127]:
#shot copy

def model_predict(txt):
    vectmp = vect.transform([txt])
    tftmp = tfidf_transformer.transform(vectmp)
    return detect_model.predict(tftmp)

In [128]:
#ดูค่า Performance

from sklearn.metrics import classification_report
print (classification_report(arr_class, all_predictions))

ValueError: Found input variables with inconsistent numbers of samples: [76617, 1]

In [129]:
#ลอง test

txt = 'Ski suits; knitted or crocheted'
test = model_predict(txt)
print(test)


['845090']


In [130]:
# ตรวจสอบความแม่นยำ

print(detect_model.score(messages_tfidf,arr_class))

0.4482164532675516


In [131]:
#Save model to file

import pickle

pickle.dump(detect_model,open("data/hs_master.pickle",'wb'))

In [132]:
#Load model from file

import pickle
detect_model = pickle.load(open("data/hs_master.pickle",'rb'))
detect_model

MultinomialNB()

Reference

- http://adataanalyst.com/scikit-learn/countvectorizer-sklearn-example/
- http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

# Countvectorizer sklearn example

- https://adataanalyst.com/scikit-learn/countvectorizer-sklearn-example/

In [1]:
import pandas as pd

df1 =pd.read_excel('data/HS-Code_target.xlsx',sheet_name='level_4')
df2 =pd.read_excel('data/HS-Code_target.xlsx',sheet_name='level_6', parse_string =['Code'])
df = pd.concat([df1,df2], axis=0) 
df.head()

,Classification,Code,Description,CodeParent,Level,isLeaf
0,H0,101,"Horses, asses, mules and hinnies; live",1,4,0
1,H0,102,Bovine animals; live,1,4,0
2,H0,103,Swine; live,1,4,0
3,H0,104,Sheep and goats; live,1,4,0
4,H0,105,"Poultry; live, fowls of the species gallus dom...",1,4,0


In [2]:
data = df[['Code','Description']]
data.head()

,Code,Description
0,101,"Horses, asses, mules and hinnies; live"
1,102,Bovine animals; live
2,103,Swine; live
3,104,Sheep and goats; live
4,105,"Poultry; live, fowls of the species gallus dom..."


In [3]:
data.describe()

,Code,Description
count,41914,41914
unique,9095,12660
top,2511,Copra
freq,10,15


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41914 entries, 0 to 31047
Data columns (total 2 columns):
Code           41914 non-null object
Description    41914 non-null object
dtypes: object(2)
memory usage: 982.4+ KB


In [ ]:
data.groupby('Code').describe()

In [ ]:
data['length'] = data['Description'].apply(len)
data.head()

In [ ]:
len(data)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data['length'].plot(bins=50,kind = 'hist')

In [ ]:
data['length'].describe()

In [ ]:
data[data['length'] ==1040]['Description'].iloc[0]

In [ ]:
import string

string.punctuation

In [ ]:
def capitalize_every_word(s):
    return s.title()

In [ ]:
s = 'whether/not stripped of bark/sapwood)/roughly squared, but not treated with paint'
a = capitalize_every_word(s)
a

In [ ]:
s = '''
 in the rough (whether/not stripped of bark/sapwood)/roughly squared, but not treated with paint, stains/other preservatives of the following tropical woods:Abura, Acajou d’Afrique, Afrormosia, Ako, Alan, Andiroba, Aningré, Avodiré, Azobé, Balau, Balsa, Bossé clair, Bosséfoncé, Cativo, Cedro, Dabema, Dark Red Meranti, Dibétou,Doussié, Framiré, Freijo, Fromager, Fuma, Geronggang,Ilomba, Imbuia, Ipé, Iroko, Jaboty, Jelutong, Jequitiba,Jongkong, Kapur, Kempas, Keruing, Kosipo, Kotibé, Koto,Light Red Meranti, Limba, Louro, Maçaranduba, Mahogany,Makoré, Mandioqueira, Mansonia, Mengkulang, MerantiBakau, Merawan, Merbau, Merpauh, Mersawa, Moabi,Niangon, Nyatoh, Obeche, Okoumé, Onzabili, Orey,Ovengkol, Pzogp. Padauk, Paldao, Palissandre de Guatemala,Palissandre de Para, Palissandre de Rio, Palissandre de Rose,Pau Amarelo, Pau Marfim, Pulai, Punah, Quaruba, Ramin,Sapelli, Saqui-Saqui, Spetir, Sipo, Sucupira, Suren, Tauari,Teak, Tiama, Tola, Virola, White Lauan, White Meranti,White Seraya, Yellow Meranti

'''
print(s.lower())
a = capitalize_every_word(s.lower())
a
